In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('sim_data_trunc.csv')

In [3]:
print(df.head())

         Date  Step  price_A      tv_A  web_A  price_B  tv_B  web_B  \
0  2022-01-02    52      5.0       0.0    0.0      5.0   0.0    0.0   
1  2022-01-09    53      5.0       0.0    0.0      5.0   0.0    0.0   
2  2022-01-16    54      5.0       0.0    0.0      5.0   0.0    0.0   
3  2022-01-23    55      5.0       0.0    0.0      5.0   0.0    0.0   
4  2022-01-30    56      5.0  125000.0    0.0      5.0   0.0    0.0   

   Total_Purchases_A  Total_Purchases_B  Total_Sales_A  Total_Sales_B  
0              618.0              243.0      3090000.0      1215000.0  
1              531.0              187.0      2655000.0       935000.0  
2              653.0              240.0      3265000.0      1200000.0  
3              521.0              207.0      2605000.0      1035000.0  
4              620.0              231.0      3100000.0      1155000.0  


In [4]:
import pandas as pd
from meridian.data import load
from meridian.model import model, spec, prior_distribution
import tensorflow_probability as tfp

# Assuming df is your DataFrame
coord_to_columns = load.CoordToColumns(
    time='Date',
    controls=['price_A', 'price_B'],
    kpi='Total_Purchases_A',  # or 'Total_Purchases_B' depending on your focus
    revenue_per_kpi='Total_Sales_A',  # or 'Total_Sales_B'
    media=['tv_A', 'web_A', 'tv_B', 'web_B'],
    media_spend=['tv_A', 'web_A', 'tv_B', 'web_B'],  # Assuming spend is same as media exposure
)

media_to_channel = {
    'tv_A': 'TV_A',
    'web_A': 'Web_A',
    'tv_B': 'TV_B',
    'web_B': 'Web_B',
}

media_spend_to_channel = {
    'tv_A': 'TV_A',
    'web_A': 'Web_A',
    'tv_B': 'TV_B',
    'web_B': 'Web_B',
}

# Load the data
data_loader = load.DataFrameDataLoader(
    df=df,
    coord_to_columns=coord_to_columns,
    kpi_type='non_revenue',  # Corrected from 'non-revenue' to 'non_revenue'
    media_to_channel=media_to_channel,
    media_spend_to_channel=media_spend_to_channel,
)
data = data_loader.load()

# Configure the model
roi_mu = 0.2
roi_sigma = 0.9
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name='ROI_M')
)
model_spec = spec.ModelSpec(prior=prior)

# Initialize and run the model
mmm = model.Meridian(input_data=data, model_spec=model_spec)
mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000)

/Users/michael/.venvs/pymmm_0_6_0/lib/python3.12/site-packages/meridian/data/load.py:1020: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[geo_column_name] = self.df[geo_column_name].replace(
/Users/michael/.venvs/pymmm_0_6_0/lib/python3.12/site-packages/meridian/data/load.py:201: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if (constants.GEO) not in self.dataset.dims.keys():
/Users/michael/.venvs/pymmm_0_6_0/lib/python3.12/site-packages/meridian/data/load.py:206: UserWarning: The `population` argument is ignored in a nationally aggregated model. It wil

In [6]:
# Extract posterior samples
posterior_samples = mmm.get_posterior_samples()

# View summary statistics
summary = mmm.summary()
print(summary)

# Visualize the results
mmm.plot_posterior_distributions()
mmm.plot_media_effects()

AttributeError: 'Meridian' object has no attribute 'get_posterior_samples'